# 9. 경쟁하며 학습하는 GAN

## 9.0. 요약

### 9.0.1. 지난장

지난 장에서는 학습된 신경망에 대해 노이즈를 주어 신뢰성 있는 결과를 얻는 것을 방해하는 적대적 공격(Adversarial Attack)에 대해 알아보았다. 적대적 공격의 목표는 사람이 눈치채지 못하지만 ML 알고리즘에는 영향을 줄 수 있는 노이즈를 만드는 것이다. 그래서 예제로 FGSM을 활용한 적대적 공격을 살펴보았다.

### 9.0.2. 이번장

이번 장에서는 적대적 공격의 개념을 반영하여 신경망의 성능을 높이는 GAN(Generative Adversarial Network)에 대해서 알아보자. GAN은 비지도학습의 일종으로 앞으로 딥러닝의 발전을 이끌 신경망 중 하나로 꼽힌다. GAN은 생성부와 판별부로 나뉘며 생성부에서는 진짜 같은 가짜를 만드는 것을 목표로 하며, 판별부는 생성부의 결과값과 실제값을 올바르게 판별하는 것을 목표로 한다. 서로 다른 두 목표를 충족시켜 가며 학습을 수행하고 예측하는 신경망을 바로 GAN이라고 한다.